# M45 Coursework
## Using AlexNet and ResNet to evaluate their effectiveness on classifying objects

## Loading Data 

In [35]:
import numpy as np

images = np.load('trnImage.npy')
labels = np.load('trnLabel_coarse.npy')
labels_fine = np.load('trnLabel_fine.npy')

test_images = np.load('tstImage.npy')
test_labels = np.load('tstLabel_coarse.npy')
test_labels_fine = np.load('tstLabel_fine.npy')

print(f'Images Shape: {images.shape}')
print(f'Train Labels Coarse Shape: {labels.shape}')

Images Shape: (32, 32, 3, 50000)
Train Labels Coarse Shape: (50000,)


# Alex Net Coarse

In [23]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(20))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization_9 (Batch (None, 8, 8, 96)          384       
_________________________________________________________________
activation_9 (Activation)    (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_10 (Batc (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_10 (Activation)   (None, 4, 4, 256)        

# Complie AlexNet

In [24]:
# Compiling the model
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])


# Change input shapes on Coarse Dataset

In [36]:
train_images_reshaped = np.transpose(images, (3, 0, 1, 2))
test_images_reshaped = np.transpose(test_images, (3, 0, 1, 2))
labels_reshaped = labels.reshape(50000, 1)

labels_fine_reshaped = labels_fine.reshape(50000, 1)


x_train = train_images_reshaped
#y_train = labels_reshaped
y_train = labels_fine_reshaped

x_test = test_images_reshaped
y_test = test_labels

#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.3)


#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 20 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 20
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)


((35000, 32, 32, 3), (35000, 100))
((15000, 32, 32, 3), (15000, 100))
((10000, 32, 32, 3), (10000, 20))


In [27]:
print(f'Images Shape: {train_images_reshaped.shape}')
print(f'Train Labels Coarse Shape: {test_images_reshaped.shape}')

Images Shape: (50000, 32, 32, 3)
Train Labels Coarse Shape: (10000, 32, 32, 3)


In [37]:
#Defining the parameters
batch_size= 100
epochs=100
learn_rate=.001

In [50]:
history = keras.models.load_model('alexNet_coarse')
print(history)

# Train model

In [31]:
#Training the model
AlexNet.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = len(x_test)//batch_size, verbose=1)


Epoch 1/100
350/350 [==============================] - 156s 442ms/step - loss: 2.6588 - accuracy: 0.1843 - val_loss: 3.0741 - val_accuracy: 0.1523
Epoch 2/100
350/350 [==============================] - 159s 455ms/step - loss: 2.3056 - accuracy: 0.2861 - val_loss: 3.1576 - val_accuracy: 0.1925
Epoch 3/100
350/350 [==============================] - 161s 461ms/step - loss: 2.1845 - accuracy: 0.3224 - val_loss: 2.4502 - val_accuracy: 0.2684
Epoch 4/100
350/350 [==============================] - 154s 441ms/step - loss: 2.0883 - accuracy: 0.3517 - val_loss: 2.7001 - val_accuracy: 0.2463
Epoch 5/100
350/350 [==============================] - 147s 420ms/step - loss: 2.0153 - accuracy: 0.3740 - val_loss: 2.2482 - val_accuracy: 0.3184
Epoch 6/100
350/350 [==============================] - 148s 422ms/step - loss: 1.9406 - accuracy: 0.3971 - val_loss: 3.1157 - val_accuracy: 0.2049
Epoch 7/100
350/350 [==============================] - 148s 423ms/step - loss: 1.8967 - accuracy: 0.4132 - val_loss: 2

In [32]:
AlexNet.save('alexNet_coarse')

INFO:tensorflow:Assets written to: alexNet_coarse\assets


In [44]:
print(AlexNet.predict(test_images_reshaped))

[[2.43103271e-03 2.69028940e-03 3.49790382e-04 ... 2.72551752e-05
  2.52320140e-04 1.88424962e-03]
 [2.16222252e-03 4.38046787e-04 7.33196503e-04 ... 1.40379707e-03
  7.08199479e-03 1.51955225e-02]
 [6.87568307e-01 5.32912761e-02 1.24701794e-04 ... 2.19036080e-03
  7.20124692e-04 5.95821766e-04]
 ...
 [4.05122182e-06 4.74672015e-05 1.19327306e-04 ... 1.09232997e-03
  4.17817546e-06 4.70710911e-05]
 [1.22256446e-04 2.36951033e-04 5.94038574e-05 ... 9.82776037e-05
  1.53403322e-04 8.84392575e-05]
 [5.14362864e-06 2.89375748e-04 9.96861339e-01 ... 3.85577034e-04
  6.31554358e-05 2.98790110e-05]]


In [46]:
import matplotlib.pyplot as plt

In [44]:
test_labels = np.load('tstLabel_coarse.npy')

In [59]:
print(test_labels.shape)
print(predictions.shape)

(10000,)
(10000, 20)


In [37]:
AlexNetFineHistory = tf.keras.models.load_model('alexNet_coarse')

In [55]:

predictions = AlexNetFineHistory.predict(test_images_reshaped)

In [39]:
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

In [45]:
print(np.unique(test_labels))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


# Alex Coarse Confusion matrix

In [61]:
predictions = AlexNetFineHistory.predict(test_images_reshaped)
cm = confusion_matrix(np.array(test_labels), np.array(predictions), labels=np.unique(test_labels))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(test_labels))
disp.plot() 

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
if self.binary:
  test_confusion = ConfusionMatrixDisplay(confusion_matrix(self.val_y.data.cpu().numpy(), np.array(predictions)),display_labels=['Benign','Attack'])
else:
  test_confusion = ConfusionMatrixDisplay(confusion_matrix(self.val_y.data.cpu().numpy(), 
                                                                 np.array(predictions)),
                                                    display_labels=['Benign','MSSQL','UDP','LDAP','NetBIOS','Portmap'])
        
print("(Top) Training Confusion Matrix")
train_confusion.plot()

In [ ]:
>>> from sklearn.datasets import make_classification
>>> from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
>>> from sklearn.model_selection import train_test_split
>>> from sklearn.svm import SVC
>>> X, y = make_classification(random_state=0)
>>> X_train, X_test, y_train, y_test = train_test_split(X, y,
...                                                     random_state=0)
>>> clf = SVC(random_state=0)
>>> clf.fit(X_train, y_train)
SVC(random_state=0)
>>> predictions = clf.predict(X_test)
>>> cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
>>> disp = ConfusionMatrixDisplay(confusion_matrix=cm,
...                               display_labels=clf.classes_)
>>> disp.plot() 

In [51]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = AlexNet.evaluate(test_images_reshaped,  test_labels_coarse, verbose=2)

print(test_acc)

TypeError: 'NoneType' object is not subscriptable

In [59]:
score = AlexNet.evaluate(test_images_reshaped, test_labels, verbose=0)
print(score)

score = AlexNet.evaluate(test_images, test_labels_fine, verbose=0)
print(score)

ValueError: in user code:

    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1233 test_function  *
        return step_function(self, iterator)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_repli

# Alex Net Fine 

In [40]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNetFine = Sequential()

#1st Convolutional Layer
AlexNetFine.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
AlexNetFine.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNetFine.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
AlexNetFine.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNetFine.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))

#4th Convolutional Layer
AlexNetFine.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))

#5th Convolutional Layer
AlexNetFine.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
AlexNetFine.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNetFine.add(Flatten())
# 1st Fully Connected Layer
AlexNetFine.add(Dense(4096, input_shape=(32,32,3,)))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNetFine.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNetFine.add(Dense(4096))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNetFine.add(Dense(1000))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('relu'))
#Add Dropout
AlexNetFine.add(Dropout(0.4))

#Output Layer
AlexNetFine.add(Dense(100))
AlexNetFine.add(BatchNormalization())
AlexNetFine.add(Activation('softmax'))

#Model Summary
AlexNetFine.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization_27 (Batc (None, 8, 8, 96)          384       
_________________________________________________________________
activation_27 (Activation)   (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_28 (Batc (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_28 (Activation)   (None, 4, 4, 256)        

In [41]:
# Compiling the model
AlexNetFine.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])


In [42]:
AlexNetFine.fit(, epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = len(x_test)//batch_size, verbose=1)


Epoch 1/100
350/350 [==============================] - 152s 432ms/step - loss: 4.2240 - accuracy: 0.0668 - val_loss: 4.0814 - val_accuracy: 0.0837
Epoch 2/100
350/350 [==============================] - 152s 434ms/step - loss: 3.6774 - accuracy: 0.1317 - val_loss: 3.8463 - val_accuracy: 0.1213
Epoch 3/100
350/350 [==============================] - 150s 429ms/step - loss: 3.4801 - accuracy: 0.1654 - val_loss: 3.9826 - val_accuracy: 0.1068
Epoch 4/100
350/350 [==============================] - 154s 440ms/step - loss: 3.3198 - accuracy: 0.1967 - val_loss: 4.0372 - val_accuracy: 0.1150
Epoch 5/100
350/350 [==============================] - 156s 446ms/step - loss: 3.1956 - accuracy: 0.2146 - val_loss: 3.9655 - val_accuracy: 0.1228
Epoch 6/100
350/350 [==============================] - 155s 442ms/step - loss: 3.1008 - accuracy: 0.2354 - val_loss: 3.9543 - val_accuracy: 0.1234
Epoch 7/100
350/350 [==============================] - 154s 439ms/step - loss: 3.0194 - accuracy: 0.2490 - val_loss: 3

In [43]:
AlexNetFine.save('alexNet_fine')

INFO:tensorflow:Assets written to: alexNet_fine\assets


In [8]:
model.save('resNet_coarse_fail')

INFO:tensorflow:Assets written to: resNet_coarse_fail\assets


In [13]:
history = tf.keras.models.load_model('resNet_coarse_fail')


NameError: name 'train_y' is not defined

In [14]:
print(history.predict(y_train))

ValueError: in user code:

    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_re

# ResNet Coarse

In [8]:
model.save('resNet_coarse')

INFO:tensorflow:Assets written to: resNet_coarse_fail\assets


In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import copy
from scipy.io import savemat,loadmat
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

import tensorflow
print('Tensorflow version = ',tensorflow.__version__)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler, History

#from tensorflow.keras import backend as K

from ResNetModel import resnet
from Utils import cutout,LR_WarmRestart,GetDataGen,plot_history

In [ ]:
#params
init_lr = 0.1
epochs = 100
batch_size = 100
My_wd=5e-4/2
resnet_width = 10
resnet_depth = 20
UseBinary=False
UseCutout=True
Loss = 'categorical_crossentropy'
Optimizer = SGD(lr=init_lr,decay=0.0, momentum=0.9, nesterov=False)
Metrics = ['accuracy']
ModelsPath = 'TrainedModels/Tensorflow.keras/'

In [ ]:
import numpy as np
import tensorflow as tf
x_train = np.load('trnImage.npy')
y_train = np.load('trnLabel_coarse.npy')
#labels_fine = np.load('trnLabel_fine.npy')

x_test = np.load('tstImage.npy')
y_test = np.load('tstLabel_coarse.npy')
#test_labels_fine = np.load('tstLabel_fine.npy')

x_train = np.transpose(x_train, (3, 0, 1, 2))
x_test = np.transpose(x_test, (3, 0, 1, 2))
y_train = y_train.reshape(50000, 1)
y_test = y_test.reshape(10000, 1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

num_classes = np.unique(y_train).shape[0]
K_train = x_train.shape[0]
input_shape = x_train.shape[1:]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#fdefine a datagen or generating training samples with flip and pad/crop augmentation, and if set to True, with cutout augmentation
dataGenerator = GetDataGen(UseCutout)

#define and compile the model
ResnetCoarsemodel = resnet(UseBinary,input_shape=input_shape, depth=resnet_depth, num_classes=num_classes,wd=My_wd,width=resnet_width)
ResnetCoarsemodel.compile(loss=Loss ,optimizer = Optimizer, metrics = Metrics)

#print  the model
ResnetCoarsemodel.summary()

#define the learnng rate schedule
steps_per_epoch = int(np.floor(K_train / batch_size))
lr_scheduler = LR_WarmRestart(nbatch=steps_per_epoch,
                              initial_lr=init_lr, min_lr=init_lr*1e-4,
                              epochs_restart = [1.0,3.0, 7.0, 15.0, 31.0, 63.0,127.0,255.0]) 

#define callbacks
history = History()
callbacks = [lr_scheduler,history]

In [30]:
ResnetCoarsemodelHistory = ResnetCoarsemodel.fit(dataGenerator.flow(x_train, y_train, batch_size=50),
                              validation_data=(x_test, y_test),
                              epochs=50,
                              verbose=1,
                              callbacks=callbacks,
                              steps_per_epoch=150)

Epoch 1/50

 Start of Epoch Learning Rate = 0.096119
150/150 [==============================] - 866s 6s/step - loss: 5.6113 - accuracy: 0.2129 - val_loss: 6.0751 - val_accuracy: 0.0500

 End of Epoch Learning Rate = 0.080978
Epoch 2/50

 Start of Epoch Learning Rate = 0.080978
150/150 [==============================] - 879s 6s/step - loss: 5.2187 - accuracy: 0.2373 - val_loss: 5.9592 - val_accuracy: 0.0500

 End of Epoch Learning Rate = 0.063602
Epoch 3/50

 Start of Epoch Learning Rate = 0.063602
150/150 [==============================] - 857s 6s/step - loss: 4.9095 - accuracy: 0.2749 - val_loss: 5.6069 - val_accuracy: 0.0500

 End of Epoch Learning Rate = 0.044156
Epoch 4/50

 Start of Epoch Learning Rate = 0.044156
150/150 [==============================] - 844s 6s/step - loss: 4.6987 - accuracy: 0.2965 - val_loss: 5.5715 - val_accuracy: 0.0500

 End of Epoch Learning Rate = 0.025600
Epoch 5/50

 Start of Epoch Learning Rate = 0.025600
150/150 [==============================] - 864s

150/150 [==============================] - 848s 6s/step - loss: 2.0117 - accuracy: 0.5420 - val_loss: 4.0534 - val_accuracy: 0.0500

 End of Epoch Learning Rate = 0.058317
Epoch 38/50

 Start of Epoch Learning Rate = 0.058317
150/150 [==============================] - 850s 6s/step - loss: 1.9601 - accuracy: 0.5561 - val_loss: 3.8614 - val_accuracy: 0.0500

 End of Epoch Learning Rate = 0.053445
Epoch 39/50

 Start of Epoch Learning Rate = 0.053445
150/150 [==============================] - 852s 6s/step - loss: 1.9337 - accuracy: 0.5568 - val_loss: 3.8406 - val_accuracy: 0.0552

 End of Epoch Learning Rate = 0.048540
Epoch 40/50

 Start of Epoch Learning Rate = 0.048540
150/150 [==============================] - 850s 6s/step - loss: 1.9200 - accuracy: 0.5665 - val_loss: 4.9649 - val_accuracy: 0.0500

 End of Epoch Learning Rate = 0.043648
Epoch 41/50

 Start of Epoch Learning Rate = 0.043648
150/150 [==============================] - 850s 6s/step - loss: 1.8681 - accuracy: 0.5765 - val_

In [ ]:
ResnetCoarsemodelHistory = ResnetCoarsemodel.fit(dataGenerator.flow(x_train, y_train, batch_size=50),
                              validation_data=(x_test, y_test),
                              epochs=50,
                              verbose=1,
                              callbacks=callbacks,
                              steps_per_epoch=50)

In [35]:
print(ResnetCoarsemodel.predict(test_images))

ValueError: in user code:

    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_re

# ResNet fine labels

In [62]:
import numpy as np
import tensorflow as tf
x_train = np.load('trnImage.npy')
y_train = np.load('trnLabel_fine.npy')
#labels_fine = np.load('trnLabel_fine.npy')

x_test = np.load('tstImage.npy')
y_test = np.load('tstLabel_fine.npy')
#test_labels_fine = np.load('tstLabel_fine.npy')

x_train = np.transpose(x_train, (3, 0, 1, 2))
x_test = np.transpose(x_test, (3, 0, 1, 2))
y_train = y_train.reshape(50000, 1)
y_test = y_test.reshape(10000, 1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

num_classes = np.unique(y_train).shape[0]
K_train = x_train.shape[0]
input_shape = x_train.shape[1:]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [63]:
#fdefine a datagen or generating training samples with flip and pad/crop augmentation, and if set to True, with cutout augmentation
dataGenerator = GetDataGen(UseCutout)

#define and compile the model
ResnetFinemodel = resnet(UseBinary,input_shape=input_shape, depth=resnet_depth, num_classes=num_classes,wd=My_wd,width=resnet_width)
ResnetFinemodel.compile(loss=Loss ,optimizer = Optimizer, metrics = Metrics)

#print  the model
ResnetFinemodel.summary()

#define the learnng rate schedule
steps_per_epoch = int(np.floor(K_train / batch_size))
lr_scheduler = LR_WarmRestart(nbatch=steps_per_epoch,
                              initial_lr=init_lr, min_lr=init_lr*1e-4,
                              epochs_restart = [1.0,3.0, 7.0, 15.0, 31.0, 63.0,127.0,255.0]) 

#define callbacks
history = History()
callbacks = [lr_scheduler,history]

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization_63 (BatchNo (None, 32, 32, 3)    12          input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_60 (Conv2D)              (None, 32, 32, 160)  4320        batch_normalization_63[0][0]     
__________________________________________________________________________________________________
batch_normalization_64 (BatchNo (None, 32, 32, 160)  320         conv2d_60[0][0]                  
____________________________________________________________________________________________

In [ ]:
ResnetFineModelHistory = ResnetFinemodel.fit(dataGenerator.flow(x_train, y_train, batch_size=50),
                              validation_data=(x_test, y_test),
                              epochs=50,
                              verbose=1,
                              callbacks=callbacks,
                              steps_per_epoch =50)

Epoch 1/50

 Start of Epoch Learning Rate = 0.100000
26/50 [==============>...............] - ETA: 1:50 - loss: 8.0615 - accuracy: 0.0254